In [10]:
# import required packages
import glob
import time
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
import keras
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score
##CNN
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier
#saving model
from keras.models import load_model
import pickle


[nltk_data] Downloading package stopwords to C:\Users\Syed
[nltk_data]     Usama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Read data as csv

In [11]:
df=pd.read_csv('sqli.csv',encoding='utf-16')

In [12]:
# vectorization of data
vectorizer = CountVectorizer( min_df=2, max_df=0.7, max_features=4096, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(df['Sentence'].values.astype('U')).toarray()



In [13]:
posts.shape

(4200, 4096)

In [14]:
posts.shape=(4200,64,64,1)
posts.shape

(4200, 64, 64, 1)

In [15]:
X=posts
y=df['Label']
# split train test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

trainX=X_train.copy()
trainX.shape=(X_train.shape[0],trainX.shape[1]*trainX.shape[2])

testX=X_test.copy()
testX.shape=(testX.shape[0],testX.shape[1]*testX.shape[2])
print(X_train.shape,trainX.shape,testX.shape,y_train.shape,y_test.shape,testX.shape)

(3360, 64, 64, 1) (3360, 4096) (840, 4096) (3360,) (840,) (840, 4096)


#### Naive Bayes

In [16]:
gnb = GaussianNB()

gnb.fit(trainX, y_train)

pred_gnb = gnb.predict(testX)

#### Support Vector Machine

In [17]:
clf = SVC(gamma='auto')
clf.fit(trainX, y_train)
pred_svm=clf.predict(testX)

#### K Nearest Neighbor

In [18]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(trainX, y_train)

pred_knn = neigh.predict(testX)

#### Decision Tree

In [19]:
dt = tree.DecisionTreeClassifier()
dt = dt.fit(trainX, y_train)

pred_dt = dt.predict(testX)

In [20]:
X_train.shape

(3360, 64, 64, 1)

#### CNN

In [21]:
model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, input_shape=(64,64,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation='sigmoid')
])



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [22]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 256)         0         
____________________________

In [23]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=128)


Train on 3360 samples, validate on 840 samples
Epoch 1/10
3360/3360 [==============================] - 60s 18ms/sample - loss: 0.2919 - acc: 0.8640 - val_loss: 0.1782 - val_acc: 0.9226
Epoch 2/10
3360/3360 [==============================] - 60s 18ms/sample - loss: 0.1364 - acc: 0.9435 - val_loss: 0.1326 - val_acc: 0.9464
Epoch 3/10
3360/3360 [==============================] - 65s 19ms/sample - loss: 0.1058 - acc: 0.9551 - val_loss: 0.1089 - val_acc: 0.9512
Epoch 4/10
3360/3360 [==============================] - 61s 18ms/sample - loss: 0.1012 - acc: 0.9563 - val_loss: 0.1154 - val_acc: 0.9476
Epoch 5/10
3360/3360 [==============================] - 54s 16ms/sample - loss: 0.0788 - acc: 0.9646 - val_loss: 0.0883 - val_acc: 0.9571
Epoch 6/10
3360/3360 [==============================] - 54s 16ms/sample - loss: 0.0667 - acc: 0.9720 - val_loss: 0.0875 - val_acc: 0.9607
Epoch 7/10
3360/3360 [==============================] - 55s 16ms/sample - loss: 0.0711 - acc: 0.9741 - val_loss: 0.0994 - val

In [24]:
pred=model.predict(X_test)

In [25]:
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [26]:
accuracy_score(y_test,pred)

0.9678571428571429

In [27]:
for i,j in zip(y_test,pred):
    print(i==j)

[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]


#### Saving Model

In [28]:
model.save('my_model_cnn.h5')
with open('vectorizer_cnn', 'wb') as fin:
    pickle.dump(vectorizer, fin)

### Confusion Matrix
#### Assumption:
1. SQLI attack type is positive

In [29]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy
def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision
def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall


In [30]:
def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [31]:
accuracy,precision,recall=confusion_matrix(y_test,pred)

In [32]:
print(" For CNN \n Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For CNN 
 Accuracy : 0.9678571428571429 
 Precision : 0.9213483146067416 
 Recall : 0.9761904761904762


In [33]:
accuracy,precision,recall=confusion_matrix(y_test,pred_svm)
print(" For SVM Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For SVM Accuracy : 0.7642857142857142 
 Precision : 1.0 
 Recall : 0.21428571428571427


In [34]:
accuracy,precision,recall=confusion_matrix(y_test,pred_knn)
print(" For KNN Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For KNN Accuracy : 0.5976190476190476 
 Precision : 0.4271186440677966 
 Recall : 1.0


In [35]:
accuracy,precision,recall=confusion_matrix(y_test,pred_dt)
print(" For Decision Tree Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For Decision Tree Accuracy : 0.8678571428571429 
 Precision : 0.6942148760330579 
 Recall : 1.0


In [36]:
accuracy,precision,recall=confusion_matrix(y_test,pred_gnb)
print(" For Naive Bayes  Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For Naive Bayes  Accuracy : 0.9773809523809524 
 Precision : 0.9298892988929889 
 Recall : 1.0


### Classifying user data

     This cell is also in file user_Data_predict.py No need to run this cell

In [37]:
mymodel = tf.keras.models.load_model('my_model_cnn.h5')
myvectorizer = pickle.load(open("vectorizer_cnn", 'rb'))

def clean_data(input_val):

    input_val=input_val.replace('\n', '')
    input_val=input_val.replace('%20', ' ')
    input_val=input_val.replace('=', ' = ')
    input_val=input_val.replace('((', ' (( ')
    input_val=input_val.replace('))', ' )) ')
    input_val=input_val.replace('(', ' ( ')
    input_val=input_val.replace(')', ' ) ')
    input_val=input_val.replace('1 ', 'numeric')
    input_val=input_val.replace(' 1', 'numeric')
    input_val=input_val.replace("'1 ", "'numeric ")
    input_val=input_val.replace(" 1'", " numeric'")
    input_val=input_val.replace('1,', 'numeric,')
    input_val=input_val.replace(" 2 ", " numeric ")
    input_val=input_val.replace(' 3 ', ' numeric ')
    input_val=input_val.replace(' 3--', ' numeric--')
    input_val=input_val.replace(" 4 ", ' numeric ')
    input_val=input_val.replace(" 5 ", ' numeric ')
    input_val=input_val.replace(' 6 ', ' numeric ')
    input_val=input_val.replace(" 7 ", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace('1234', ' numeric ')
    input_val=input_val.replace("22", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace(" 200 ", ' numeric ')
    input_val=input_val.replace("23 ", ' numeric ')
    input_val=input_val.replace('"1', '"numeric')
    input_val=input_val.replace('1"', '"numeric')
    input_val=input_val.replace("7659", 'numeric')
    input_val=input_val.replace(" 37 ", ' numeric ')
    input_val=input_val.replace(" 45 ", ' numeric ')

    return input_val


def predict_sqli_attack():
    
    repeat=True
    
    beautify=''
    for i in range(20):
        beautify+= "="

    print(beautify) 
    input_val=input("Give me some data to work on : ")
    print(beautify)

    
    if input_val== '0':
        repeat=False
    
    

    input_val=clean_data(input_val)
    input_val=[input_val]



    input_val=myvectorizer.transform(input_val).toarray()
    
    input_val.shape=(1,64,64,1)

    result=mymodel.predict(input_val)


    print(beautify)
    
    
    if repeat == True:
        
        if result>0.5:
            print("ALERT :::: This can be SQL injection")


        elif result<=0.5:
            print("It seems to be safe")
            
        print(beautify)
            
        predict_sqli_attack()
            
    elif repeat == False:
        print( " Good Bye ")

 

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [38]:
# check for sql injection cheat sheet
predict_sqli_attack()

Give me some data to work on : attack 9/11
It seems to be safe
Give me some data to work on : give me glass of water
It seems to be safe
Give me some data to work on : 1==1
ALERT :::: This can be SQL injection
Give me some data to work on : select *
ALERT :::: This can be SQL injection
Give me some data to work on : 0
 Good Bye 
